In [1]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_02'

# 01 - Get Datasets


**Data will be Imported from a CSV which is expected to be in this format**
- sub column contents MUST match the names of the neuroimaging files above. 
    - ID column 
```
+-----+----------------------------+--------------+--------------+--------------+
| sub | Nifti_File_Path            | Indep. Var.  | Covariate_N  | Dataset      |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 1            |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 1            |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 2            |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3            |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

In [2]:
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/metadata/master_list_v3.csv'
sheet = None # Set to None if CSV

In [4]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
display(data_df)

,Dataset,Subject,Nifti_File_Path,age,sex,diagnosis,overall_cognition,lesion_size,current_memory,percent_decline_memory,percent_memory_improvement
0,adni_memory,002_S_0295,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,84.898630,NaN,Normal,3.00,NaN,1.0,10.000000,-10.000000
1,adni_memory,002_S_0413,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,76.397260,NaN,Normal,3.33,NaN,1.0,10.000000,-10.000000
2,adni_memory,002_S_0559,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,79.410959,NaN,Normal,6.00,NaN,2.0,20.000000,-20.000000
3,adni_memory,002_S_0619,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,77.512329,NaN,Alzheimer,19.33,NaN,10.0,100.000000,-100.000000
4,adni_memory,002_S_0685,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,89.698630,NaN,Normal,3.67,NaN,3.0,30.000000,-30.000000
...,...,...,...,...,...,...,...,...,...,...,...
2558,fornix_memory,146,/Volumes/OneTouch/datasets/AD_dataset/146/stim...,76.000000,NaN,NaN,4.00,NaN,17.0,54.545455,-54.545455
2559,fornix_memory,147,/Volumes/OneTouch/datasets/AD_dataset/147/stim...,59.000000,NaN,NaN,3.00,NaN,22.0,4.761905,-4.761905
2560,fornix_memory,148,/Volumes/OneTouch/datasets/AD_dataset/148/stim...,51.000000,NaN,NaN,3.50,NaN,40.0,207.692308,-207.692308
2561,fornix_memory,149,/Volumes/OneTouch/datasets/AD_dataset/149/stim...,77.000000,NaN,NaN,5.00,NaN,19.0,90.000000,-90.000000


**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [5]:
drop_list = ['Nifti_File_Path', 'percent_memory_improvement', 'overall_cognition']

In [6]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
display(data_df)

,Dataset,Subject,Nifti_File_Path,age,sex,diagnosis,overall_cognition,lesion_size,current_memory,percent_decline_memory,percent_memory_improvement
0,adni_memory,002_S_0295,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,84.898630,NaN,Normal,3.00,NaN,1.0,10.000000,-10.000000
1,adni_memory,002_S_0413,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,76.397260,NaN,Normal,3.33,NaN,1.0,10.000000,-10.000000
2,adni_memory,002_S_0559,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,79.410959,NaN,Normal,6.00,NaN,2.0,20.000000,-20.000000
3,adni_memory,002_S_0619,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,77.512329,NaN,Alzheimer,19.33,NaN,10.0,100.000000,-100.000000
4,adni_memory,002_S_0685,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,89.698630,NaN,Normal,3.67,NaN,3.0,30.000000,-30.000000
...,...,...,...,...,...,...,...,...,...,...,...
2558,fornix_memory,146,/Volumes/OneTouch/datasets/AD_dataset/146/stim...,76.000000,NaN,NaN,4.00,NaN,17.0,54.545455,-54.545455
2559,fornix_memory,147,/Volumes/OneTouch/datasets/AD_dataset/147/stim...,59.000000,NaN,NaN,3.00,NaN,22.0,4.761905,-4.761905
2560,fornix_memory,148,/Volumes/OneTouch/datasets/AD_dataset/148/stim...,51.000000,NaN,NaN,3.50,NaN,40.0,207.692308,-207.692308
2561,fornix_memory,149,/Volumes/OneTouch/datasets/AD_dataset/149/stim...,77.000000,NaN,NaN,5.00,NaN,19.0,90.000000,-90.000000


**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

In [9]:
column = 'diagnosis'
condition = 'equal'
value = 'MCI'

In [10]:
data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
data_df

,Dataset,Subject,Nifti_File_Path,age,sex,diagnosis,overall_cognition,lesion_size,current_memory,percent_decline_memory,percent_memory_improvement
3,adni_memory,002_S_0619,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,77.512329,NaN,Alzheimer,19.33,NaN,10.0,100.000000,-100.000000
5,adni_memory,002_S_0729,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,65.224658,NaN,Alzheimer,6.67,NaN,8.0,80.000000,-80.000000
7,adni_memory,002_S_0816,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,70.838356,F,Alzheimer,16.00,NaN,8.0,80.000000,-80.000000
8,adni_memory,002_S_0938,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,82.276712,F,Alzheimer,21.67,NaN,10.0,100.000000,-100.000000
9,adni_memory,002_S_0954,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,69.452055,NaN,Alzheimer,10.67,NaN,9.0,90.000000,-90.000000
...,...,...,...,...,...,...,...,...,...,...,...
2558,fornix_memory,146,/Volumes/OneTouch/datasets/AD_dataset/146/stim...,76.000000,NaN,NaN,4.00,NaN,17.0,54.545455,-54.545455
2559,fornix_memory,147,/Volumes/OneTouch/datasets/AD_dataset/147/stim...,59.000000,NaN,NaN,3.00,NaN,22.0,4.761905,-4.761905
2560,fornix_memory,148,/Volumes/OneTouch/datasets/AD_dataset/148/stim...,51.000000,NaN,NaN,3.50,NaN,40.0,207.692308,-207.692308
2561,fornix_memory,149,/Volumes/OneTouch/datasets/AD_dataset/149/stim...,77.000000,NaN,NaN,5.00,NaN,19.0,90.000000,-90.000000


**Standardize Data**
- Enter Columns you Don't want to standardize into a list

In [11]:
# Remove anything you don't want to standardize
cols_not_to_standardize = ['Dataset', 'Subject']
group_col = 'Dataset' 

In [12]:
data_df = cal_palm.standardize_columns(cols_not_to_standardize, group_col)
data_df

Unable to standardize column Nifti_File_Path for group Dataset
Unable to standardize column sex for group Dataset
Unable to standardize column diagnosis for group Dataset
Unable to standardize column Nifti_File_Path for group Dataset
Unable to standardize column sex for group Dataset
Unable to standardize column Nifti_File_Path for group Dataset
Unable to standardize column Nifti_File_Path for group Dataset
Unable to standardize column sex for group Dataset
Unable to standardize column Nifti_File_Path for group Dataset
Unable to standardize column Nifti_File_Path for group Dataset
Unable to standardize column Nifti_File_Path for group Dataset


,Dataset,Subject,Nifti_File_Path,age,sex,diagnosis,overall_cognition,lesion_size,current_memory,percent_decline_memory,percent_memory_improvement
3,adni_memory,002_S_0619,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,0.246314,NaN,Alzheimer,0.761459,NaN,1.170312,1.170312,-1.165854
5,adni_memory,002_S_0729,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,-1.265430,NaN,Alzheimer,-1.219342,NaN,0.318243,0.318243,-0.322384
7,adni_memory,002_S_0816,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,-0.574780,F,Alzheimer,0.240443,NaN,0.318243,0.318243,-0.322384
8,adni_memory,002_S_0938,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,0.832473,F,Alzheimer,1.127579,NaN,1.170312,1.170312,-1.165854
9,adni_memory,002_S_0954,/Volumes/OneTouch/datasets/adni/neuroimaging/a...,-0.745336,NaN,Alzheimer,-0.593496,NaN,0.744277,0.744277,-0.744119
...,...,...,...,...,...,...,...,...,...,...,...
2558,fornix_memory,146,/Volumes/OneTouch/datasets/AD_dataset/146/stim...,1.048470,NaN,NaN,-0.461554,NaN,-0.979918,0.278570,-0.278570
2559,fornix_memory,147,/Volumes/OneTouch/datasets/AD_dataset/147/stim...,-1.131835,NaN,NaN,-1.047654,NaN,-0.448797,-0.700641,0.700641
2560,fornix_memory,148,/Volumes/OneTouch/datasets/AD_dataset/148/stim...,-2.157861,NaN,NaN,-0.754604,NaN,1.463238,3.290872,-3.290872
2561,fornix_memory,149,/Volumes/OneTouch/datasets/AD_dataset/149/stim...,1.176723,NaN,NaN,0.124546,NaN,-0.767470,0.975939,-0.975939


# 02 - Import the Data into DataFrames

In [15]:
import os
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
from calvin_utils.statistical_utils.regression_utils import RegressOutCovariates
import tensorly as tl
from tensorly.regression import CPRegressor
import statsmodels.api as sm
from calvin_utils.file_utils.import_functions import GiiNiiFileImport

class DatasetNiftiImporter(GiiNiiFileImport):
    def __init__(self, df, dataset_col, nifti_col, indep_var_col, covariate_cols, out_dir, mask_path=None, regression_method='tensor'):
        self.df = df
        self.dataset_col = dataset_col
        self.nifti_col = nifti_col
        self.indep_var_col = indep_var_col
        self.covariate_cols = covariate_cols
        self.out_dir = out_dir
        self.mask_path = mask_path
        self.data_dict = {}
        self.regression_method = regression_method
        self._prepare_data_dict()
        
    def _prep_dataset_dir(self, dataset):
        dataset_dir = os.path.join(self.out_dir, 'tmp', dataset)
        os.makedirs(dataset_dir, exist_ok=True)
        return dataset_dir

    def _prepare_data_dict(self):
        for dataset in self.df[self.dataset_col].unique():
            print("Importing dataset: ", dataset)
            dataset_df = self.df[self.df[self.dataset_col] == dataset]
            dataset_df.columns = self.df.columns
            dataset_dir = self._prep_dataset_dir(dataset)

            self.data_dict[dataset] = {
                'niftis': pd.DataFrame(),
                'indep_var': pd.DataFrame(),
                'covariates': pd.DataFrame()
            }

            nifti_paths = dataset_df[self.nifti_col].tolist()
            nifti_importer = GiiNiiFileImport(import_path=None, file_column=None, file_pattern=None)
            self.data_dict[dataset]['niftis'] = nifti_importer.import_matrices(nifti_paths)
            self.data_dict[dataset]['indep_var'] = dataset_df.loc[:, [self.indep_var_col]]
            self.data_dict[dataset]['covariates'] = dataset_df.loc[:, self.covariate_cols]

            self._process_dataset(dataset, dataset_dir)

    def _add_intercept(self, covariates_arr):
        intercept = np.ones((covariates_arr.shape[0], 1)) # shape (samples, 1)
        return np.hstack((intercept, covariates_arr)) # shape (samples, N_covariates + 1)
    
    def _tensor_regression(self, endog, exog):
        # Ensure endog and exog are tensors
        endog = tl.tensor(endog)
        exog = tl.tensor(exog)

        # Perform tensor regression using CP decomposition
        model = CPRegressor(weight_rank=1)
        model.fit(exog, endog)
        residuals = endog - model.predict(exog)
        return residuals

    def _ols_regression(self, endog, exog):
        residuals = np.zeros_like(endog)
        for i in tqdm(range(endog.shape[1]), desc='OLS Regression'):
            model = sm.OLS(endog[:, i], exog).fit()
            residuals[:, i] = model.resid
        return residuals
    
    def _run_regression(self, masked_niftis_array, indep_var_array, covariates_array):
        print(masked_niftis_array.shape, indep_var_array.shape, covariates_array.shape)
        if self.regression_method == 'tensor':
            # Perform tensor regression
            nifti_residuals = self._tensor_regression(masked_niftis_array, covariates_array)
            indep_residuals = self._tensor_regression(indep_var_array, covariates_array)
        elif self.regression_method == 'ols':
            # Perform OLS regression
            nifti_residuals = self._ols_regression(masked_niftis_array, covariates_array)
            indep_residuals = self._ols_regression(indep_var_array, covariates_array)
        else:
            raise ValueError("Invalid regression method. Choose 'tensor' or 'ols'.")
        return nifti_residuals, indep_residuals
    
    def _save_residuals(self, dataset_dir, nifti_residuals_arr, indep_residuals_arr, covariates_arr):
        np.save(os.path.join(dataset_dir, 'niftis.npy'), nifti_residuals_arr)
        np.save(os.path.join(dataset_dir, 'indep_var.npy'), indep_residuals_arr)
        np.save(os.path.join(dataset_dir, 'covariates.npy'), covariates_arr)
        
    def _process_dataset(self, dataset, dataset_dir):
        niftis = self.data_dict[dataset]['niftis'] # Get the nifti
        _, _, masked_niftis = self.mask_dataframe(niftis, mask_path=self.mask_path) # mask the nifti 
        masked_niftis_arr = masked_niftis.values.T                          # shape (samples, voxels)
        indep_var_arr = self.data_dict[dataset]['indep_var'].values         # shape (samples, 1)
        covariates_arr = self.data_dict[dataset]['covariates'].values       # shape (samples, N_covariates)
        covariates_arr = self._add_intercept(covariates_arr)                # shape (samples, N_covariates+1)
        
        nifti_residuals, indep_residuals = self._run_regression(masked_niftis_arr, indep_var_arr, covariates_arr)

        self._save_residuals(dataset_dir, nifti_residuals, indep_residuals, covariates_arr)

    def cleanup(self):
        tmp_dir = os.path.join(self.out_dir, 'tmp')
        shutil.rmtree(tmp_dir)

In [16]:
dataset_col = 'Dataset'
nifti_path_col = 'Nifti_File_Path'
indep_var_col = 'percent_memory_improvement'
covariate_cols = ['overall_cognition']

In [18]:
# from calvin_utils.file_utils.import_functions import DatasetNiftiImporter
data_importer = DatasetNiftiImporter(df=data_df, dataset_col=dataset_col, nifti_col=nifti_path_col, indep_var_col=indep_var_col, covariate_cols=covariate_cols, out_dir=out_dir, regression_method='tensor')

Importing dataset:  adni_memory
(20, 225222) (20, 1) (20, 2)

Converged in 39 iterations
Importing dataset:  corbetta_memory
(20, 225222) (20, 1) (20, 2)
Importing dataset:  grafmann_memory
(20, 225222) (20, 1) (20, 2)

Converged in 21 iterations
Importing dataset:  manitoba_memory
(8, 225222) (8, 1) (8, 2)

Converged in 5 iterations

Converged in 12 iterations
Importing dataset:  queensland_memory
55
['/Volumes/OneTouch/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs/sub-LeadDBS029/stimulations/MNI152NLin2009bAsym/gs_20200826142724/GSP 1000 (Yeo 2011)_Full Set (Yeo 2011)/vat_seed_compound_fMRI_efield_func_seed_T.nii', '/Volumes/OneTouch/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs/sub-LeadDBS030/stimulations/MNI152NLin2009bAsym/gs_20200826142724/GSP 1000 (Yeo 2011)_Full Set (Yeo 2011)/vat_seed_compound_fMRI_efield_func_seed_T.nii', '/Volumes/OneTouch/datasets/Queensland_PD_DBS_STN/BIDSdata/derivatives/leaddbs/sub-LeadDBS031/stimulations/MNI152NLin2009bAsym/g

# 04 - Begin Analysis